In [45]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [46]:
df_clean = pd.read_csv('data/df_clean.csv')

df_clean = df_clean.assign(
    sous_poids = (df_clean['bmi']<18.5).astype(int), 
    poids_normal = ((df_clean['bmi']>= 18.5) & (df_clean['bmi']<24.9)).astype(int),
    surpoids = ((df_clean['bmi']>=24.9) & (df_clean['bmi']<29.9)).astype(int),
    obésité =((df_clean['bmi']>= 29.9)).astype(int),
    # obésité_morbide = ((df_clean['bmi']>=34.9)).astype(int)
)



# selection des features et de la cible
X = df_clean.drop('charges', axis=1)

y = df_clean.charges

X


,age,sex,bmi,children,smoker,region,sous_poids,poids_normal,surpoids,obésité
0,19,female,27.900,0,yes,southwest,0,0,1,0
1,18,male,33.770,1,no,southeast,0,0,0,1
2,28,male,33.000,3,no,southeast,0,0,0,1
3,33,male,22.705,0,no,northwest,0,1,0,0
4,32,male,28.880,0,no,northwest,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,0,0,0,1
1333,18,female,31.920,0,no,northeast,0,0,0,1
1334,18,female,36.850,0,no,southeast,0,0,0,1
1335,21,female,25.800,0,no,southwest,0,0,1,0


In [47]:
# Création du train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.85, random_state=42, stratify=X['smoker'])

In [48]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from feature_engine.discretisation import ArbitraryDiscretiser

# Séparation des colonnes numériques et catégorielles

cat_col = ['region']

num_col = X.select_dtypes(include = 'number').columns
num_col = num_col[num_col != 'bmi']

ordinal_col= ['sex', 'smoker']

#Transformation de BMI avec FunctionTransformer


# pipeline pour les valeurs numériques
# cela permet de créer une succession d'étapes (ici il n'y en a qu'une)


my_num_pipe = make_pipeline(StandardScaler())

my_cat_pipe = make_pipeline(OneHotEncoder())

my_ordi_pipe = make_pipeline(OneHotEncoder(drop= 'if_binary'))

                            
# ColumnTransformer permet de faire des opérations sur des sélections de colonnes

preprocessor = make_column_transformer(
                                       (my_ordi_pipe,ordinal_col),  
                                       (my_cat_pipe,cat_col),
                                       
                                       remainder='passthrough'
                                       )


final_preprocessor = make_pipeline(preprocessor,StandardScaler(),PolynomialFeatures(degree=2),)


# affichage

tab = final_preprocessor.fit(X_train)

cols = tab.get_feature_names_out()

df_transform = pd.DataFrame(final_preprocessor.fit_transform(X_train), columns=cols)
df_transform.head()

,1,pipeline-1__sex_male,pipeline-1__smoker_yes,pipeline-2__region_northeast,pipeline-2__region_northwest,pipeline-2__region_southeast,pipeline-2__region_southwest,remainder__age,remainder__bmi,remainder__children,...,remainder__sous_poids^2,remainder__sous_poids remainder__poids_normal,remainder__sous_poids remainder__surpoids,remainder__sous_poids remainder__obésité,remainder__poids_normal^2,remainder__poids_normal remainder__surpoids,remainder__poids_normal remainder__obésité,remainder__surpoids^2,remainder__surpoids remainder__obésité,remainder__obésité^2
0,1.0,-1.017763,-0.507965,1.760989,-0.558371,-0.622139,-0.561084,-1.508889,0.086969,2.452595,...,0.016100,0.055603,0.077216,-0.114121,0.192025,0.266668,-0.394122,0.370326,-0.547323,0.808917
1,1.0,-1.017763,1.968638,-0.567863,-0.558371,-0.622139,1.782266,1.766367,0.078750,0.768613,...,0.016100,0.055603,0.077216,-0.114121,0.192025,0.266668,-0.394122,0.370326,-0.547323,0.808917
2,1.0,0.982547,-0.507965,-0.567863,1.790924,-0.622139,-0.561084,-1.437688,-0.490810,-0.915369,...,0.016100,0.055603,-0.208508,0.141079,0.192025,-0.720091,0.487222,2.700326,-1.827074,1.236220
3,1.0,0.982547,-0.507965,-0.567863,1.790924,-0.622139,-0.561084,-1.437688,-2.192918,-0.915369,...,62.111111,-3.453534,-4.795971,-8.762592,0.192025,0.266668,0.487222,0.370326,0.676612,1.236220
4,1.0,0.982547,1.968638,-0.567863,-0.558371,-0.622139,1.782266,-0.583273,-0.808877,1.610604,...,0.016100,0.055603,-0.208508,0.141079,0.192025,-0.720091,0.487222,2.700326,-1.827074,1.236220


In [49]:
final_preprocessor

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  ['sex', 'smoker']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['region'])])),
                ('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures())])

In [50]:
from sklearn.model_selection import GridSearchCV

model= make_pipeline(final_preprocessor, ElasticNet(max_iter=1000000,random_state=42))

# Définir les valeurs à tester pour alpha et l1_ratio

param_grid = {'elasticnet__alpha': np.arange(1,200,1),   
              'elasticnet__l1_ratio':[1]}      


# creation de l'objet GridSearch

grid_search = GridSearchCV(model,param_grid, cv=5)

# recherche des hyperparamètres et Afficher les meilleures valeurs trouvées

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)

print("Score : ", best_model.score(X_test, y_test))

# calcul RSME

rmse= np.sqrt(mean_squared_error(y_test,best_model.predict(X_test)))
print("RSME: ",rmse)


Meilleurs paramètres :  {'elasticnet__alpha': 130, 'elasticnet__l1_ratio': 1}
Score :  0.9175935712563795
RSME:  3455.7188270098304


## Regression Linéaire 

In [51]:
model= make_pipeline(final_preprocessor, LinearRegression())
model.fit(X_train,y_train)
model.score(X_train,y_train)
print("R2 (coefficient de détermination): ",model.score(X_test,y_test))




R2 (coefficient de détermination):  0.9088897081004832


## Regression Lasso

In [52]:
model= make_pipeline(final_preprocessor,Lasso(max_iter=1000000))

param_grid={'lasso__alpha':np.arange(0.1,100,10)}

grid_search = GridSearchCV(model,param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)


Meilleurs paramètres :  {'lasso__alpha': 90.1}


0.9180290475285556

## Regression Ridge

In [53]:
model= make_pipeline(final_preprocessor,Ridge(max_iter=1000000))

param_grid={'ridge__alpha':np.arange(0.1,100,10)}

grid_search = GridSearchCV(model,param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Meilleurs paramètres : ",grid_search.best_params_)

# Utiliser le modèle avec les meilleures valeurs
best_model = grid_search.best_estimator_

best_model.score(X_test, y_test)


Meilleurs paramètres :  {'ridge__alpha': 40.1}


0.9128145970482042